# Thesis

## Install Library

In [ ]:
!pip install roboflow
!pip install ultralytics

## Import Library

In [1]:
import time
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
from roboflow import Roboflow
from ultralytics import YOLO

## Download Dataset

In [4]:
# Dataset Object Detection
rf = Roboflow(api_key="m1DZYxXInBZk9zMy3jDT")
project = rf.workspace("abiya").project("plant-pathology-anotation")
version = project.version(7)
dataset = version.download("yolov11", location="dataset/objectDetection/")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dataset/objectDetection/ in yolov11:: 100%|██████████| 6950/6950 [00:01<00:00, 4524.27it/s]


In [5]:
# Dataset Mask Lesi
rf = Roboflow(api_key="m1DZYxXInBZk9zMy3jDT")
project = rf.workspace("abiya").project("plant-pathology-anotation")
version = project.version(10)
dataset = version.download("yolov11", location="dataset/maskLesi/")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dataset/maskLesi/ in yolov11:: 100%|██████████| 130/130 [00:00<00:00, 5405.57it/s]


## Training Object Detection

In [5]:
# Global Variables
epochs = 2
imgsz = 640
model_variants_objectDetection = ['n', 's', 'm', 'l']
dataset_objectDetection = "dataset/objectDetection/data.yaml"
output_csv_objectDetection = "results/objectDetection/csv/"
output_runs_objectDetection = "results/objectDetection/runs/"
output_plot_objectDetection = "results/objectDetection/plot/"

# Memastikan folder output ada
os.makedirs(output_csv_objectDetection, exist_ok=True)
os.makedirs(output_runs_objectDetection, exist_ok=True)
os.makedirs(output_plot_objectDetection, exist_ok=True)

In [ ]:
# Inisialisasi file ringkasan
summary_train_file = os.path.join(output_csv_objectDetection, "final_train_summary.csv")
summary_val_file = os.path.join(output_csv_objectDetection, "final_val_summary.csv")

# Header ringkasan
train_header = [
    "variant", "train_time_h", "train_time_m", "train_time_s"
]
val_header = [
    "variant", "precision", "recall", "mAP50", "mAP50-95",
]

# Inisialisasi file ringkasan
with open(summary_train_file, mode="w", newline="") as f_train, \
     open(summary_val_file, mode="w", newline="") as f_val:
    csv.writer(f_train).writerow(train_header)
    csv.writer(f_val).writerow(val_header)

In [ ]:
# Loop tiap varian model
for variant in model_variants_objectDetection:
    model_name = f"yolo11{variant}.pt"
    train_name = f"train_{variant}"
    project_path = f"{output_runs_objectDetection}detect/{train_name}"
    model = YOLO(model_name)

    # Train
    start_time = time.time()
    results = model.train(
        data=dataset_objectDetection,
        epochs=epochs,
        imgsz=imgsz,
        project=f"{output_runs_objectDetection}/detect",
        name=train_name,
        verbose=False
    )
    end_time = time.time()

    # Simpan waktu training
    elapsed = end_time - start_time
    hours = int(elapsed // 3600)
    minutes = int((elapsed % 3600) // 60)
    seconds = int(elapsed % 60)

    # Simpan ke summary_train_file
    with open(summary_train_file, mode="a", newline="") as f_train:
        csv.writer(f_train).writerow([variant, hours, minutes, seconds])

    # === Simpan log setiap epoch ===
    results_csv_path = os.path.join(project_path, "results.csv")
    if os.path.exists(results_csv_path):
        df = pd.read_csv(results_csv_path)
        df.to_csv(os.path.join(output_csv_objectDetection, f"train_logs_{variant}.csv"), index=False)

    # === Evaluasi / Validasi ===
    best_model_path = os.path.join(project_path, "weights", "best.pt")
    model = YOLO(best_model_path)
    val_name = f"val_{variant}"
    metrics = model.val(
        data=dataset_objectDetection,
        imgsz=imgsz,
        split='test',
        project=f"{output_runs_objectDetection}/detect",
        name=val_name,
        verbose=False
    )

    # Simpan ke summary_val_file
    with open(summary_val_file, mode="a", newline="") as f_val:
        writer = csv.writer(f_val)
        results_dict = metrics.results_dict
        writer.writerow([
            variant,
            round(results_dict['metrics/precision(B)'], 4),
            round(results_dict['metrics/recall(B)'], 4),
            round(results_dict['metrics/mAP50(B)'], 4),
            round(results_dict['metrics/mAP50-95(B)'], 4),
        ])


New https://pypi.org/project/ultralytics/8.3.162 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.137 🚀 Python-3.10.16 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/objectDetection/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_n, nbs=64, nms=False, o

train: Scanning /home/abiyamf/Thesis/dataset/objectDetection/train/labels.cache... 1557 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1557/1557 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1635.3±1415.0 MB/s, size: 159.4 KB)


val: Scanning /home/abiyamf/Thesis/dataset/objectDetection/valid/labels.cache... 173 images, 0 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]


Plotting labels to results/objectDetection/runs/detect/train_n/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to results/objectDetection/runs/detect/train_n
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

train: Scanning /home/abiyamf/Thesis/dataset/objectDetection/train/labels.cache... 1557 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1557/1557 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2034.3±1857.9 MB/s, size: 159.4 KB)


val: Scanning /home/abiyamf/Thesis/dataset/objectDetection/valid/labels.cache... 173 images, 0 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]


Plotting labels to results/objectDetection/runs/detect/train_m/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to results/objectDetection/runs/detect/train_m
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2      7.94G      1.035      3.167      1.594         54        640:   5%|▌         | 5/98 [02:48<49:47, 32.12s/it]  

In [ ]:
# Perbandingan Metrik antar Varian
metrics = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5-0.95",
    "time": "time",
}

# Buat dictionary file
files = {v: os.path.join(output_csv_objectDetection, f"train_logs_{v}.csv") for v in model_variants_objectDetection}

# Loop setiap metrik dan buat plot perbandingan
for metric_col, metric_label in metrics.items():
    plt.figure()
    for variant, file_path in files.items():
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if metric_col in df.columns:
                plt.plot(df["epoch"], df[metric_col], label=f"YOLOv11-{variant}")

    plt.xlabel("Epoch")
    plt.ylabel(metric_label)
    plt.title(f"Perbandingan {metric_label} antar Varian")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    safe_name = metric_col.replace("/", "_")
    plt.savefig(os.path.join(output_plot_objectDetection, f"compare_{safe_name}.png"))
    plt.close()

## Training Mask Daun

## Training Mask Lesi